In [1]:
!nvidia-smi


Fri Jul 28 12:36:18 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   63C    P0               5W /  50W |      5MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
###yolov5 install
import os
HOME = os.getcwd()
%cd {HOME}
!git clone https://github.com/ultralytics/yolov5
%cd {HOME}/yolov5
!pip install -r requirements.txt
from IPython import display
display.clear_output()

In [3]:
##YOLOv8 install
!pip install ultralytics
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.143 🚀 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)
Setup complete ✅ (8 CPUs, 15.5 GB RAM, 31.7/72.3 GB disk)


In [20]:
from ultralytics import YOLO
import numpy as np
model = YOLO('yolov8s.pt')

In [5]:
!pip list


Package                   Version
------------------------- --------------------
anyio                     3.7.1
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
arrow                     1.2.3
asttokens                 2.2.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.12.1
backcall                  0.2.0
beautifulsoup4            4.12.2
bleach                    6.0.0
boltons                   23.0.0
brotlipy                  0.7.0
certifi                   2023.7.22
cffi                      1.15.1
charset-normalizer        2.0.4
cmake                     3.27.0
comm                      0.1.3
conda                     23.7.2
conda-content-trust       0.1.3
conda-libmamba-solver     23.5.0
conda-package-handling    2.1.0
conda_package_streaming   0.8.0
contourpy                 1.1.0
cryptography              39.0.1
cycler                    0.11.0
debugpy                   1.6.7
decorator                 5.1.1
defused

In [2]:
## Video_Info
import supervision as sv
vid = '/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/Data/Crowd_Count/Sample_Videos/CCTV_Ireland_Sample_480.mp4'
sv.VideoInfo.from_video_path(vid)

VideoInfo(width=852, height=480, fps=30, total_frames=1200)

In [7]:
# Extract First Frame to draw polygon zone manually
import cv2

generator = sv.get_video_frames_generator(vid)
iterator = iter(generator)
frame = next(iterator)

cv2.imwrite("frame.jpg", frame)

True

In [3]:
import numpy as np
# initiate polygon zone
## Use this website to mark pixels ---> https://pixspy.com
import torch
import cv2

model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')



colors = sv.ColorPalette.default()
polygons = [
    np.array([[246,276],
              [248,296],
              [427,294],
              [391,272],
              [310,269] 
    ], np.int32),
    np.array([[144,208],
              [209,285],
              [0,326],
              [0,205]
    ], np.int32),
    np.array([[236,300],
              [148,323],
              [166,392],
              [358,389],
              [370,361],
              [284,306]   
    ], np.int32),
    np.array([[371,364],
              [407,345],
              [477,340],
              [486,385],
              [432,417],
              [368,385],
              [366,366]
    ], np.int32),
    np.array([[464,287],
              [551,281],
              [576,327],
              [485,340],
              [462,293]
    ], np.int32),
    np.array([[576,356],
              [523,398],
              [709,479],
              [762,478],
              [773,441],
              [680,404],
              [601,368]
    ], np.int32)
]
video_info = sv.VideoInfo.from_video_path(vid)

zones = [
    sv.PolygonZone(
        polygon=polygon, 
        frame_resolution_wh=video_info.resolution_wh
    )
    for polygon
    in polygons
]

zone_annotators = [
    sv.PolygonZoneAnnotator(
        zone=zone, 
        color=colors.by_idx(index), 
        thickness=2,
        text_thickness=1,
        text_scale=0.4,
        text_padding=1
    )
    for index, zone
    in enumerate(zones)
]

box_annotators = [
    sv.BoxAnnotator(
        color=colors.by_idx(index), 
        thickness=1, 
        text_thickness=1, 
        text_scale=0.35,
        text_padding=1
        )
    for index
    in range(len(polygons))
]


def process_frame(frame: np.ndarray, i) -> np.ndarray:
    # detect
    print(i)
    results = model(frame, size=1280)
    detections = sv.Detections.from_yolov5(results)
    detections = detections[(detections.class_id == 0) & (detections.confidence > 0.5)]

    for zone, zone_annotator, box_annotator in zip(zones, zone_annotators, box_annotators):
        mask = zone.trigger(detections=detections)
        detections_filtered = detections[mask]
        # labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
        frame = box_annotator.annotate(scene=frame, detections=detections_filtered, skip_label=True)
        frame = zone_annotator.annotate(scene=frame)

    sv.show_frame_in_notebook(frame,(16, 16))
    return frame


# generator = sv.get_video_frames_generator(vid)
# iterator = iter(generator)
# frame = next(iterator)
# 
# results = model(frame, imgsz=1280)
# detections = sv.Detections.from_yolov8(results)
# detections = detections[(detections.class_id == 0) & (detections.confidence > 0.5)]
# 
# for zone, zone_annotator, box_annotator in zip(zones, zone_annotators, box_annotators):
#     mask = zone.trigger(detections=detections)
#     detections_filtered = detections[mask]
#     frame = box_annotator.annotate(scene=frame, detections=detections_filtered, skip_label=True)
#     frame = zone_annotator.annotate(scene=frame)
# 
# sv.show_frame_in_notebook(frame,(16,16))


sv.process_video(source_path=vid, target_path=f"Output2.mp4", callback=process_frame)



from IPython import display
display.clear_output()

In [15]:
cap = cv2.VideoCapture(vid)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        break
        
    frame = process_frame(frame = frame)

    cv2.imshow('Output', frame)
    # result.write(frame)
    # if detection.__len__() == 15:
    #     break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 736x1280 36 persons, 2 bicycles, 1 truck, 1 stop sign, 5 backpacks, 1 handbag, 1 skateboard, 1 chair, 6 potted plants, 78.0ms
Speed: 6.9ms preprocess, 78.0ms inference, 3.3ms postprocess per image at shape (1, 3, 736, 1280)
/home/manz/miniconda3/lib/python3.10/site-packages/supervision/detection/core.py:175: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(index, np.ndarray) and index.dtype == np.bool:


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [1]:
import supervision as sv

sv.VideoInfo.from_video_path('/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/Data/Crowd_Count/Sample_Videos/market-square.mp4').resolution_wh

(2160, 3840)

In [3]:
import cv2

vid = cv2.VideoCapture('/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/Data/Crowd_Count/Sample_Videos/market-square.mp4')

ret, frame = vid.read()

(frame.shape[1], frame.shape[0])

(2160, 3840)